#### Setup the notebook

In [1]:
#************************************************************************************************
# Imports
#************************************************************************************************
import os
from dotenv import load_dotenv
import pandas as pd
from openai import AzureOpenAI
import json
import time

# Import the assistant tools
from tools import get_list_of_dockets

#************************************************************************************************
# Load environment variables from a .env file and set local variables
#************************************************************************************************
load_dotenv()

AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
RA_ASSISTANT_ID = "asst_Ga0X4abWQFGSvfDnS0TY0LcP"


#************************************************************************************************
# Create client instance of the AzureOpenAI class
#************************************************************************************************
AOAI = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )

#************************************************************************************************
# Setup utility functions
#************************************************************************************************

# Pretty printing JSON
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Pretty printing messages
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = AOAI.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)  
        time.sleep(0.5)
    return run

def submit_message(assistant_id, thread, user_message):
    AOAI.beta.threads.messages.create(thread_id=thread.id, role="user", content=user_message)
    return AOAI.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

def get_response(thread):
    return AOAI.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = AOAI.beta.threads.create()
    run = submit_message(RA_ASSISTANT_ID, thread, user_input)
    return thread, run



In [2]:
thread, run = create_thread_and_run("What proposed regulations from 2024 have to do with ITAR.")
run = wait_on_run(run, thread)
run.status

'requires_action'

In [ ]:
show_json(run)

In [3]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: get_list_of_dockets
Function Arguments:


{'searchTerm': 'ITAR',
 'afterPostedDate': '2024-01-01',
 'beforePostedDate': '2024-12-31'}

In [4]:
function_to_call = globals()[name]
result = function_to_call(**arguments)

print("Function Output:")
print(result)

Making API Call to: https://api.regulations.gov/v4/documents
With parameters: {'api_key': '4p2Hpwlq1SJ5kOhayfhqeI0D1RtDOo70d8azejwL', 'filter[documentType]': 'Proposed Rule', 'sort': '-postedDate', 'filter[searchTerm]': 'ITAR', 'filter[postedDate][le]': '2024-12-31', 'filter[postedDate][ge]': '2024-01-01'}
API Call Successful!
Function Output:
{'data': [{'id': 'DOS-2024-0013-0001', 'type': 'documents', 'attributes': {'documentType': 'Proposed Rule', 'lastModifiedDate': '2024-05-01T13:18:56Z', 'highlightedContent': '', 'frDocNum': '2024-08829', 'withdrawn': False, 'agencyId': 'DOS', 'commentEndDate': '2024-06-01T03:59:59Z', 'title': 'International Traffic in Arms Regulations: Exemption for Defense Trade and Cooperation Among Australia, the United Kingdom, and the United States', 'postedDate': '2024-05-01T04:00:00Z', 'docketId': 'DOS-2024-0013', 'subtype': None, 'commentStartDate': '2024-05-01T04:00:00Z', 'openForComment': True, 'objectId': '090000648652cd63'}, 'links': {'self': 'https:/

In [5]:
run = AOAI.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(result),
        }
    ],
)
show_json(run)

{'id': 'run_GFbDQErf62GFbx0fc2AOoMf9',
 'assistant_id': 'asst_Ga0X4abWQFGSvfDnS0TY0LcP',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1714694116,
 'expires_at': 1714694716,
 'failed_at': None,
 'file_ids': [],
 'instructions': "**Persona**\nYou are an expert rulemaking assistant skilled at responding to questions about government regulations and using the regulations.gov API. You write concisely and clearly, in plain language, responding directly to users.\n\n**Situation**\nUsers will ask you questions about proposed government rules. You will use your tools and knowledge to provide thorough and well-supported answers.\n\n**Tools Available**\n### get_list_of_dockets - Use this tool to get a list of dockets from regulations.gov.\n- Parameters: \n    - agencyId: Filters results for the agency acronym specified in the value. Example: 'EPA'\n    - searchTerm: Filters results on the given term.\n    - postedDate: Filters results for dockets posted on the date. Format of date

In [6]:
run = wait_on_run(run, thread)
run.status

'completed'

In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

In [ ]:
responses = get_data_from_last_10K_filing_by_section(ticker="MSFT",section_path="Role_DisclosureEARNINGSPERSHARETables")
print("Responses:", responses)

In [ ]:
run = AOAI.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

In [7]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What proposed regulations from 2024 have to do with ITAR.
assistant: In 2024, there are proposed regulations related to the International Traffic in Arms Regulations (ITAR) as follows:

1. **International Traffic in Arms Regulations: Exemption for Defense Trade and Cooperation Among Australia, the United Kingdom, and the United States**
   - **Docket ID**: DOS-2024-0013
   - **Posted Date**: 2024-05-01
   - **Comment End Date**: 2024-06-01
   - **Agency**: Department of State (DOS)
   - [Link to document](https://api.regulations.gov/v4/documents/DOS-2024-0013-0001)

2. **International Traffic in Arms Regulations: Registration Fees**
   - **Docket ID**: DOS-2023-0034
   - **Posted Date**: 2024-04-24
   - **Comment End Date**: 2024-06-11
   - **Agency**: Department of State (DOS)
   - [Link to document](https://api.regulations.gov/v4/documents/DOS-2023-0034-0001)

3. **Defense Federal Acquisition Regulation Supplement: Assuring Integrity of Overseas Fuel Supplies**
   - 